In [1]:
# If you don't already have it...
#import sys
#!{sys.executable} -m pip install nflows
#!{sys.executable} -m pip install uproot

### Extracting and inverting transformations

Up until now, I've managed to train a flow to go from a Gaussian to the LHCb simulated IP distribution of Z->mu mu decays. 

It'll work just as well from a Gaussian to the data shape.

If this were to be used in an analysis, we'd need to tie these together into 1 2-step transformation.

Then we need to be able to extract the transformation as a 1-to-1 function that we can apply, rather than a sampling.

First of all, let's use a v.simple test transform of e.g. 1D Gaussian -> Gaussian to work out how to string 2 transforms together, and then extract a transformation function. We could then apply the transformation function on another sample to check that it works fine.  